In [2]:
import pandas as pd
import numpy as np
import os

# ✅ Use a dictionary (league name : file path)
file_paths = {
    "ligue1": "Transfermarkt-data/valeurs_marchandes_ligue1.csv",
    "premier_league": "Transfermarkt-data/valeurs_marchandes_premier_league.csv",
    "serieA": "Transfermarkt-data/valeurs_marchandes_serieA.csv",
    "bundesliga": "Transfermarkt-data/valeurs_marchandes_bundesliga.csv",
    "liga": "Transfermarkt-data/valeurs_marchandes_liga.csv"
}

# Function to convert 'Valeur marchande' to euros
def convert_market_value(value):
    if pd.isna(value):
        return np.nan
    value = value.replace("€", "").replace(".", "").strip()
    try:
        if "mio" in value:
            return float(value.replace("mio", "").replace(",", ".").strip()) * 1_000_000
        elif "K" in value:
            return float(value.replace("K", "").replace(",", ".").strip()) * 1_000
        else:
            return float(value.replace(",", "."))
    except:
        return np.nan

# Load, clean, and store DataFrames
dfs = {}
for league, path in file_paths.items():
    try:
        df = pd.read_csv(path)

        # Convert market value column
        df["Valeur marchande (euros)"] = df["Valeur marchande"].apply(convert_market_value)

        dfs[league] = df
        print(f"✅ {league}: Loaded and converted market values.")
    except Exception as e:
        print(f"⚠️ Error processing {league}: {e}")


✅ ligue1: Loaded and converted market values.
✅ premier_league: Loaded and converted market values.
✅ serieA: Loaded and converted market values.
✅ bundesliga: Loaded and converted market values.
✅ liga: Loaded and converted market values.


In [5]:
dfs["premier_league"].head()


,Saison,Équipe,Nom,Valeur marchande,Valeur marchande (euros)
0,2020/2021,manchester-city,Ederson,"50,00 mio. €",50000000.0
1,2020/2021,manchester-city,Zack Steffen,"6,00 mio. €",6000000.0
2,2020/2021,manchester-city,Scott Carson,300 K €,300000.0
3,2020/2021,manchester-city,James Trafford,-,NaN
4,2020/2021,manchester-city,Rúben Dias,"75,00 mio. €",75000000.0


In [6]:
for league, df in dfs.items():
    if "Valeur marchande" in df.columns:
        df.drop(columns=["Valeur marchande"], inplace=True)
        print(f"🧹 Dropped 'Valeur marchande' from {league}")
    else:
        print(f"ℹ️ Column 'Valeur marchande' not found in {league}")

🧹 Dropped 'Valeur marchande' from ligue1
🧹 Dropped 'Valeur marchande' from premier_league
🧹 Dropped 'Valeur marchande' from serieA
🧹 Dropped 'Valeur marchande' from bundesliga
🧹 Dropped 'Valeur marchande' from liga


In [7]:
dfs["premier_league"].head()


,Saison,Équipe,Nom,Valeur marchande (euros)
0,2020/2021,manchester-city,Ederson,50000000.0
1,2020/2021,manchester-city,Zack Steffen,6000000.0
2,2020/2021,manchester-city,Scott Carson,300000.0
3,2020/2021,manchester-city,James Trafford,NaN
4,2020/2021,manchester-city,Rúben Dias,75000000.0


In [8]:
for league, df in dfs.items():
    print(f"\n📊 Missing values in {league.upper()} (%):")
    
    # Calculate % of missing values
    missing_percent = df.isnull().mean() * 100
    
    # Show only columns with missing values
    missing_percent = missing_percent[missing_percent > 0].sort_values(ascending=False)
    
    if missing_percent.empty:
        print("✅ No missing values.")
    else:
        print(missing_percent.round(2))



📊 Missing values in LIGUE1 (%):
Valeur marchande (euros)    3.83
dtype: float64

📊 Missing values in PREMIER_LEAGUE (%):
Valeur marchande (euros)    8.3
dtype: float64

📊 Missing values in SERIEA (%):
Valeur marchande (euros)    2.92
dtype: float64

📊 Missing values in BUNDESLIGA (%):
Valeur marchande (euros)    2.08
dtype: float64

📊 Missing values in LIGA (%):
Valeur marchande (euros)    5.7
dtype: float64


In [9]:
# drop rows with missing valeur marchande euros value : 
for league, df in dfs.items():
    initial_shape = df.shape
    df.dropna(subset=["Valeur marchande (euros)"], inplace=True)
    dfs[league] = df  # update cleaned DataFrame
    final_shape = df.shape
    print(f"🧹 {league}: Dropped {initial_shape[0] - final_shape[0]} rows with missing market value")


🧹 ligue1: Dropped 113 rows with missing market value
🧹 premier_league: Dropped 274 rows with missing market value
🧹 serieA: Dropped 71 rows with missing market value
🧹 bundesliga: Dropped 53 rows with missing market value
🧹 liga: Dropped 174 rows with missing market value


In [11]:
import os

# Create output directory if it doesn't exist
output_dir = "Transfermarkt-data/cleaned_final_data"
os.makedirs(output_dir, exist_ok=True)

# Save each DataFrame to CSV
for league, df in dfs.items():
    output_path = os.path.join(output_dir, f"{league}_cleaned.csv")
    df.to_csv(output_path, index=False)
    print(f"✅ Saved: {output_path}")


✅ Saved: Transfermarkt-data/cleaned_final_data/ligue1_cleaned.csv
✅ Saved: Transfermarkt-data/cleaned_final_data/premier_league_cleaned.csv
✅ Saved: Transfermarkt-data/cleaned_final_data/serieA_cleaned.csv
✅ Saved: Transfermarkt-data/cleaned_final_data/bundesliga_cleaned.csv
✅ Saved: Transfermarkt-data/cleaned_final_data/liga_cleaned.csv
